In [1]:
import os
import sys

from typing import Tuple, List, Optional
from qiskit import QuantumCircuit

script_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(script_dir)

from scripts.agents.greedy_refiner  import GreedyRefiner


GRIDSYNTH = "gridsynth"
SOLOVAY_KITAEV = "solovay_kitaev"

KAK = "kak"
BQSKit = "bqskit"


In [2]:
from qiskit.circuit.random import random_circuit

my_circuit = random_circuit(num_qubits=5, depth=12, max_operands=2, seed=42)
circuit_name = "my_random_circuit"
my_circuit.draw(fold=-1)

┌─────────────────────────┐                  ┌───┐      ┌────────────┐            ┌───┐                                                                         ┌───────────────────┐         ┌───┐      ┌────────────────────────────┐       ┌────────────┐                                                                                                                    
q_0: ┤0                        ├──────────────────┤ H ├──────┤ Rx(4.6795) ├────────────┤ H ├──────────────────────────────────────────────■───────────────────■──────┤ U2(4.1589,3.4999) ├─────────┤ I ├──────┤ U3(0.55072,0.74145,6.0438) ├───────┤ Rz(4.7659) ├────────────────────────────────────────────────────────────────────────■───────────────────────────────■───────────
     │                         │┌────────────┐    └─┬─┘      └───┬───┬────┘            ├───┴┐          ┌───┐   ┌──────┐                   │                 ┌─┴──┐   └───┬────────────┬──┘    ┌────┴───┴─────┐└────────────────────────────┘       └────────────┘                     ┌──────────────────────────┐                       │             ┌───────────┐     │           
q_1: ┤                         ├┤ Rz(2.3298) ├──────┼────────────┤ X ├─────────────────┤ √X ├──────────┤ Y ├───┤ √Xdg ├───────────────────┼─────────────────┤ Sx ├───────┤ Ry(1.8447) ├───────┤1             ├────────────────────────────────────────────────────────────────■───────┤ U3(2.8637,1.2715,1.9224) ├──────────────■────────┼─────────────┤ P(6.0228) ├─────┼───────────
     │  (XX-YY)(4.939,0.80496) │└────────────┘      │       ┌────┴───┴─────┐┌──────────┴────┴─────────┐├───┤   └──────┘                   │                 ├───┬┘       └───┬────┬───┘       │              │        ┌───────────┐                ┌────────────┐             │       └────────────┬─────────────┘┌─────┐┌──────┴──────┐ │          ┌──┴───────────┴───┐ │           
q_2: ┤                         ├─■──────────────────■───────┤1             ├┤ U(1.4257,4.2086,2.7467) ├┤ H ├──────────────────────────────┼─────────────────┤ H ├────────────┤ √X ├───────────┤  Ryy(2.9021) ├────────┤ P(1.0133) ├────────────────┤ U1(5.7088) ├─────────────┼────────────────────┼──────────────┤ Sdg ├┤ Ry(0.26147) ├─┼──────────┤ R(2.1794,3.7128) ├─┼───────────
     │                         │ │            ┌────────────┐│              │└──────────┬───┬──────────┘├───┤┌────────────┐                │                 └─┬─┘            └────┘           │              │        └───────────┘                └────────────┘       ┌─────┴──────┐             │              └─────┘└─────────────┘ │P(3.1038) └──────────────────┘ │P(0.14328) 
q_3: ┤1                        ├─┼────────────┤ Ry(2.2276) ├┤  Rxx(4.2917) ├───────────┤ Y ├───────────┤ Y ├┤ Rx(2.8835) ├────────────────┼───────────────────■───────────────────────────────┤0             ├──────────────────────────────────────────────────────────┤ Ry(4.3964) ├─────────────■─────────────────────────────────────■───────────────────────────────■───────────
     └─────────────────────────┘ │U1(2.8299)  ├────────────┤│              │       ┌───┴───┴────┐      └───┘└────────────┘┌───────────────┴────────────────┐┌───┐ ┌──────────────────────────┐└┬────────────┬┘           ┌──────┐           ┌──────────────────────────┐└────────────┘                                                                                               
q_4: ────────────────────────────■────────────┤ Ry(4.7632) ├┤0             ├───────┤ Rx(5.2319) ├─────────────────────────┤ U(4.9444,4.1774,4.4307,4.9055) ├┤ Y ├─┤ U3(1.4699,0.36633,1.768) ├─┤ Rz(3.1482) ├────────────┤ √Xdg ├───────────┤ U3(3.6393,1.1107,5.3823) ├─────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                              └────────────┘└──────────────┘       └────────────┘                         └────────────────────────────────┘└───┘ └──────────────────────────┘ └────────────┘            └──────┘           └──────────────────────────┘

In [3]:
from scripts.agents.base_agent import CliffordTSynthesizer

synthesis_config = {
    "EPS": 0.01,
    "DEPTH": 2,
    "MODE1": GRIDSYNTH,
    "MODE2": KAK
}


synthesizer = CliffordTSynthesizer(mode1=synthesis_config["MODE1"],
                                   mode2=synthesis_config["MODE2"],
                                   epsilon=synthesis_config["EPS"],
                                   recursion_degree=synthesis_config["DEPTH"])
synthesized_circ, t_counts = synthesizer(my_circuit)
print(f"Initial T-count: {sum(t_counts)}")

Initial T-count: 614


In [6]:
model_config = {
    "ONLY_INTERACTIONS": True,
}

my_sequence = [(0, 1), (2, 3), (1, 4)]

optimizer = GreedyRefiner(
    synthesis_config=synthesis_config,
    model_config=model_config,
    target_circuit_and_name=(my_circuit, circuit_name),
    initial_sequence=my_sequence
)

results = optimizer.optimize(verbose=True)
print(f"Refined T-count: {results[1]} with sequence {results[0]}")


Target circuit name: my_random_circuit
Number of qubits in target circuit: 5
Preprocessing done. Circuit cleaned.
First synthesis completed in 1.83 seconds.
========== Initial Circuit Statistics ==========
Number of qubits:       5
Number of gates:        43
Initial T-count:        614
Time taken:             1.83 seconds
Possible merges:        8
Merge strategy:         8 / 10 qubit pairs selected (interacting only)


Starting GreedyRefiner with 4 stages...

=== Stage 1/4: prefix = [] ===


Optimizing: 9eval [00:02,  3.46eval/s]                     

Added merge (1, 3), new T-count: 514


Optimizing: 16eval [00:04,  4.87eval/s]

Added merge (0, 3), new T-count: 476


...

Added merge (0, 3), new T-count: 504


Optimizing: 17eval [00:02,  8.01eval/s]

Added merge (1, 2), new T-count: 478


Added merge (0, 4), new T-count: 466
No improving merge found. Terminating stage.


========== Optimization Report ==========
Applied merges: [(0, 1), (1, 3), (0, 3), (0, 4)]
Final T-count: 466
Final Fidelity: 0.996221
Time taken: 0.27 minutes

Greedy refinement complete. Total duration: 14.37 s
Refined T-count: 466 with sequence [(0, 1), (1, 3), (0, 3), (0, 4)]


([(0, 1), (1, 3), (0, 3), (0, 4)], 466)
